## Get envirionment variables

In [97]:
import os
from dotenv import find_dotenv, dotenv_values

keys = list(dotenv_values(find_dotenv('.env')).items())
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] = keys[0][1]
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY'] = keys[1][1]
POLYGON_API_KEY = os.environ['POLYGON_API_KEY'] = keys[2][1]

## Install Required Libraries

In [98]:
!pip install langchain_core langchain_openai langchain_community langsmith openai tiktoken cohere lxml polygon-api-client -qU


[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


## Set up tool belt

In [99]:
from langchain_community.tools.polygon import PolygonAggregates, PolygonFinancials, PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain.tools import tool
import datetime

@tool
def get_datetime() -> str:
    """Get the current date and time in YYYY-MM-DD HH:MM:SS format. This is useful during normal trading hours to get the most recent stock price."""
    return str(datetime.datetime.now())

@tool
def get_date() -> str:
    """Get the current date in YYYY-MM-DD format. This is useful for getting the current date if the current time is outside of normal trading hours."""
    return str(datetime.datetime.now()).split(" ")[0]

@tool
def get_time() -> str:
    """Get the current time in HH:MM:SS format. This is useful for getting the current time during normal trading hours."""
    return str(datetime.datetime.now()).split(" ")[0]

api_wrapper = PolygonAPIWrapper(polygon_api_key=POLYGON_API_KEY)

tool_belt = [
    DuckDuckGoSearchRun(),
    PolygonAggregates(api_wrapper=api_wrapper),
    PolygonFinancials(api_wrapper=api_wrapper),
    PolygonTickerNews(api_wrapper=api_wrapper),
    get_datetime,
    get_date,
    get_time
]

## Set up tool executor

In [100]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

## Set up model

In [101]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

## Set up function calling

In [102]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## Set up agent state

In [103]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Create nodes

In [104]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

In [105]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

In [106]:
workflow.set_entry_point("agent")

In [107]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

In [108]:
workflow.add_edge("action", "agent")

In [109]:
app = workflow.compile()

In [110]:
def print_messages(messages):
  next_is_tool = False
  initial_query = True
  for message in messages["messages"]:
    if "function_call" in message.additional_kwargs:
      print()
      print(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
      next_is_tool = True
      continue
    if next_is_tool:
      print(f"Tool Response: {message.content}")
      next_is_tool = False
      continue
    if initial_query:
      print(f"Initial Query: {message.content}")
      print()
      initial_query = False
      continue
    print()
    print(f"Agent Response: {message.content}")

In [115]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What was the closing price of apple on May 1st, 2024?")]}

messages = app.invoke(inputs)

print_messages(messages)

Initial Query: What was the closing price of apple on May 1st, 2024?


Tool Call - Name: get_date + Query: {}
Tool Response: 2024-07-16 19:03:36.721992

Tool Call - Name: polygon_aggregates + Query: {"ticker":"AAPL","timespan":"day","timespan_multiplier":1,"from_date":"2024-05-01","to_date":"2024-05-01"}
Tool Response: [{"v": 48416441.0, "vw": 170.7483, "o": 169.58, "c": 169.3, "h": 172.705, "l": 169.11, "t": 1714536000000, "n": 648581}]

Agent Response: The closing price of Apple (AAPL) on May 1st, 2024, was $169.30.


## Set up vector store

In [112]:
from langchain_community.vectorstores import FAISS
